### Alice möchte Bob 10000 satoshis senden welche Bob anschließend wieder ausgibt. Bob gibt Alice dabei verschiedene Möglichkeiten wie die Satoshis für ihn gelocked werden.

Die ersten Möglichkeit ist das Versenden der Satoshis an einen öffentlichen Schlüssel welcher durch einen von Bob kontrollierten privaten Schlüssel erzeugt wurde. Dieser Fall wird **P2PK-Transaktion** genannt.

In [31]:
from io import BytesIO

from src.transaction import CTx, CTxIn, CTxOut
from src.script import Script, p2pk_script
from src.ecdsa import PrivateKey

**1.** In der ersten Transaktion werden von Alice die 10000 Satoshis an den von Bob kontrollierten öffentlichen Schlüssel versendet. Um das notwendige Transaktionsformular ausfüllen zu können, benötigt Alice folgende Informationen. 

In [32]:
alice_private_key = 123
alice_previous_tx_id_to_spent = 'e0daceeaefa40ac7f3ff5298b9c3023a269a20a7aef38426d351af0cc3609649'
alice_previous_tx_index_to_spent = 0
alice_previous_script_pub_key_to_spent = '76a9148f39b19cb47396288b17c3a9f019119a140ba40088ac'
alice_amount_to_spent = 10000
bob_public_key = PrivateKey(321).get_public_key()

**1.1** Nun kann Alice die Lockingtransaktion erzeugen. Um die Satoshis an Bob's öffentlichen Schlüssel zu senden, muss Alice im Output der Transaktion ein scriptPubKey der Form: <Bob's public Key> + CHECKSIG generieren.

In [33]:
transaction_input = CTxIn(bytes.fromhex(alice_previous_tx_id_to_spent), alice_previous_tx_index_to_spent, script_sig=Script())

alice_script_pubkey = p2pk_script(bob_public_key.sec_format())
transaction_output = CTxOut(alice_amount_to_spent, alice_script_pubkey)

alice_locking_transaction = CTx(1, [transaction_input], [transaction_output], 0xffffffff, is_testnet=True, is_segwit=False)

print(f"Alice unsignierte P2PK Lockingtransaktion: {alice_locking_transaction.serialize_transaction().hex()}")

Alice unsignierte P2PK Lockingtransaktion: 0100000001499660c30caf51d32684f3aea7209a263a02c3b99852fff3c70aa4efeacedae00000000000ffffffff0110270000000000002321038b4544fc1fdfa06e456c1115a1dc831c85e7f1c5e620eca51c20802d36a4bc6bacffffffff


**1.1.1** Nun muss Alice die Lockingtransaktion noch signieren. D.h. sie muss nachweisen, dass sie aus der vorhergehenden Transaktion berechtigt ist die 10000 Satoshis auszugeben.

In [34]:
alice_script_sig = Script().parse_script(BytesIO(bytes.fromhex(f"{hex(len(alice_previous_script_pub_key_to_spent)//2)[2:]}{alice_previous_script_pub_key_to_spent}")))
alice_locking_transaction.sign_transaction(0, alice_private_key, alice_script_sig)

print(f"Alice signierte P2PK Lockingtransaktion: {alice_locking_transaction.serialize_transaction().hex()}")

Alice signierte P2PK Lockingtransaktion: 0100000001499660c30caf51d32684f3aea7209a263a02c3b99852fff3c70aa4efeacedae0000000006a473044022039f41ee6c9c93e5df7bc4c32f1af54dca5932c782efd65c2c3bef58e599b456c022055014a214c95d859669b93820f05f4788ef6deddd1005d0a56a37984958228eb012103a598a8030da6d86c6bc7f2f5144ea549d28211ea58faa70ebf4c1e665c1fe9b5ffffffff0110270000000000002321038b4544fc1fdfa06e456c1115a1dc831c85e7f1c5e620eca51c20802d36a4bc6bacffffffff


**2.** Nun möchte Bob die an seinen öffentlichen Schlüssel gesendeten Satoshis weiter versenden. Dazu muss Bob im Inputfeld (in dem auf den eben von Alice erzeugten Output referenziert wird), zum Beweis das er  berechtigt ist aen Output auszugeben lediglich eine Signatur mit Bob's privatem Schlüssel stehen. Der Einfachheit halber sendet Bob die neue Transaktion an den selben public Key wie die Alice. **Bitte macht das NIEMALS NIEMALS NIEMALS für eine echte Transaktion auf der Mainchain!!!!!**
Damit Bob das notwendige Transaktionsformular ausfüllen kann benötigt er folgende Informationen.

In [35]:
bob_private_key = 123
bob_previous_tx_id_to_spent = 'f60044de9eadb365c03d47907c200705194b2d9bbdf307d398f234143de3f291'
bob_previous_tx_index_to_spent = 0
bob_previous_script_pub_key_to_spent = alice_script_pubkey
bob_amount_to_spent = 9000

**2.1** Nun kann Bob die Transaktion erstellen um die eben von Alice erhaltenen Satoshis weiter zu versenden.

In [36]:
transaction_input = CTxIn(bytes.fromhex(bob_previous_tx_id_to_spent), bob_previous_tx_index_to_spent, script_sig=Script())

script_pubkey = p2pk_script(bob_public_key.sec_format())
transaction_output = CTxOut(bob_amount_to_spent, script_pubkey)

bob_spending_transaction = CTx(1, [transaction_input], [transaction_output], 0xffffffff, is_testnet=True, is_segwit=False)

print(f"Bob unsignierte P2PK Spendingtransaktion: {bob_spending_transaction.serialize_transaction().hex()}")


Bob unsignierte P2PK Spendingtransaktion: 010000000191f2e33d1434f298d307f3bd9b2d4b190507207c90473dc065b3ad9ede4400f60000000000ffffffff0128230000000000002321038b4544fc1fdfa06e456c1115a1dc831c85e7f1c5e620eca51c20802d36a4bc6bacffffffff


**2.2** Nun kann Bob diese Transaktion mit seinem privaten Schlüssle signieren und versenden.

In [37]:
bob_spending_transaction.sign_transaction(0, bob_private_key, bob_previous_script_pub_key_to_spent)

print(f"Bob signierte P2PK Spendingtransaktion: {bob_spending_transaction.serialize_transaction().hex()}")

Bob signierte P2PK Spendingtransaktion: 010000000191f2e33d1434f298d307f3bd9b2d4b190507207c90473dc065b3ad9ede4400f6000000006c493046022100cd7244cfe52d1dbd96f4ce77201d9f03e4713377f8c647630c7fb741cc9d6052022100e70e1b96cd92e5cf1baefcb93cd0e4c236ecb39328854f0d3c3482557f8cc9d3012103a598a8030da6d86c6bc7f2f5144ea549d28211ea58faa70ebf4c1e665c1fe9b5ffffffff0128230000000000002321038b4544fc1fdfa06e456c1115a1dc831c85e7f1c5e620eca51c20802d36a4bc6bacffffffff


now let's try this on bitcoin regtest
1. bitcoind -regtest -daemon -fallbackfee=0.0000025
2. bcr createwallet alice
3. bcr createwallet bob
4. bob's wallet loaded: bcr getnewaddress legacy bcr -rpcwallet=bob getnewaddress "" legacy result: muSAzMRK75CzLUCCFjy6TLgVr5vQ1ufkxF
5. now we need bob's private key associated with this address bcr -rpcwallet=bob dumpprivkey muSAzMRK75CzLUCCFjy6TLgVr5vQ1ufkxF, result is in wif-format: cSoDYWXxTwTbnWBpTsgWiaJbD4ZTLpJ51nHppZRFKtCK418ERJEo
6. now we need to convert bob's private key in wif to int format result: 70391367530871559720919750735257129489246656383889830217409950893950252925507
